#  normalize NOAA DSDP taxa list
## 1-96 taxa

Cleanup the normalized taxa list from the eODP researchers.

In [1]:
import sys
import csv
import glob
import os
import requests
import re
import time 

sys.path.append('../../')
import pandas as pd
import numpy as np

# import db 
import scripts.normalize_taxa as nt
from config import OUTPUT_DIR, CLEAN_DATA_DIR, RAW_DATA_DIR
from scripts.pbdb import get_parent_taxa, PBDB_TAXA_ID, PBDB_TAXA_NAME
from scripts.shared_utils import (
    log_df
)
import scripts.pbdb as pbdb

In [152]:

# lims_taxa_path = OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_list_{date}.csv'
# lims_crosswalk_path = OUTPUT_DIR/'taxa'/'LIMS'/f'taxa_crosswalk_{date}.csv'
# lims_genus_path = OUTPUT_DIR/'taxa'/'LIMS'/f'genera_pbdb_{date}.csv'




date='2022-12-01'

# metadata_1_96_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_dsdp_files.csv'
# PI_noaa_1_96_taxa_path =  RAW_DATA_DIR/'PI_processed_files'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
# noaa_1_96_crosswalk_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_crosswalk_{date}.csv"
# noaa_1_96_taxa_file = OUTPUT_DIR/'taxa'/'NOAA'/f"taxa_list_{date}.csv"
old_species_path = OUTPUT_DIR/'taxa'/'NOAA'/f'species_101_210_{date}.csv'
old_genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_101_210_{date}.csv'
old_higher_path = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_101_210_{date}.csv'



date = '2023-06-26'
initial_input_file = RAW_DATA_DIR/'PI_processed_files'/f'NOAA_taxa_lists_taxa_list_{date}.csv'
input_file = OUTPUT_DIR/'taxa'/'NOAA'/f"PI_normalized_taxa_list_with_pbdb_{date}.csv"


# metadata_path = OUTPUT_DIR/'metadata'/'NOAA'/'noaa_janus_iodp_files.csv'
# noaa_101_210_taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_101_210_{date}.csv'
# # unapproved_taxa_path = OUTPUT_DIR/'taxa'/'draft'/'NOAA'/f'taxa_unapproved_101_210_{date}.csv'
species_path = OUTPUT_DIR/'taxa'/'NOAA'/f'species_101_210_{date}.csv'
genus_path = OUTPUT_DIR/'taxa'/'NOAA'/f'genus_101_210_{date}.csv'
higher_path = OUTPUT_DIR/'taxa'/'NOAA'/f'higher_taxa_101_210_{date}.csv'




In [62]:
def add_genus_species(taxa_df):
    taxa_df.loc[~taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True).fillna(False), 'genus species']=taxa_df['genus name'] + ' ' + taxa_df['species name']
    # taxa_df.loc[taxa_df['species name'].isna(), 'genus species']=taxa_df['genus name'] 
    # taxa_df.loc[taxa_df['species name'].str.contains('spp\.|sp\..*?', regex=True).fillna(False), 'genus species']=taxa_df['genus name']
    taxa_df['genus species'] = taxa_df['genus species'].str.replace('(?)', '', regex=False)
    taxa_df['genus species'] = taxa_df['genus species'].str.replace('?', '', regex=False)

    taxa_df['genus species'] = taxa_df['genus species'].str.strip()

## fix incorect pbdb_taxon_id

incorported  pbdb_taxon_id that the PIs corrected into the taxalist.


In [222]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)
# 8906

(8906, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [223]:
PI_df.columns

Index(['taxon_group', 'verbatim_name', 'name', 'Comment',
       'Notes (change to Internal only notes?)',
       'Any taxon above genus modifier', 'Any taxon above genus',
       'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name',
       'species modifier', 'species name', 'subspecies modifier',
       'subspecies name', 'non-taxa descriptor', 'comments', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'Corrections to pbdb_taxon',
       'species_taxon_id', 'species_taxon_name', 'genus_taxon_id',
       'genus_taxon_name', 'family_taxon_id', 'family_taxon_name',
       'order_taxon_id', 'order_taxon_name', 'subclass_taxon_id',
       'subclass_taxon_name', 'class_taxon_id', 'class_taxon_name',
       'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id',
       'kingdom_taxon_name', 'unranked clade_taxon_id',
       'unranked clade_taxon_name'],
      dtype='object')

In [224]:
cols = [
    'Corrections to pbdb_taxon',
    'pbdb_taxon_id'
]
tmp = PI_df[PI_df['Corrections to pbdb_taxon'].notna()].copy()

tmp[cols].drop_duplicates()

,Corrections to pbdb_taxon,pbdb_taxon_id
849,445341,207720
850,445341,26514
1076,414258,303675
1390,468292,277953
1622,434997,206223


In [225]:
correct_col = 'Corrections to pbdb_taxon'
pbdb.fix_pbdb_id(PI_df, '445341', 445341, correct_col)
pbdb.fix_pbdb_id(PI_df, '414258', 414258, correct_col)
pbdb.fix_pbdb_id(PI_df, '468292', 468292, correct_col)
pbdb.fix_pbdb_id(PI_df, '434997', 434997, correct_col)

445341
414258
468292
434997


In [226]:
del PI_df['corrected']

In [227]:
PI_df.shape

(8906, 39)

In [230]:
PI_df.to_csv(input_file, index=True)


## update species list

PIs changed the taxa names for many taxa that did not have pbdb data; search pbdb for these revised taxa names

In [60]:
old_species_df = pd.read_csv(old_species_path, dtype=str)
log_df(old_species_df)
# (5509, 19)

(5509, 19)


,taxon_group,genus species,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,Dinoflagellates/Acritarchs/Prasinophytes,Aandalusiella ivoirensis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium cardioforme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dinoflagellates/Acritarchs/Prasinophytes,Abratopdinium kerguelense,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dinoflagellates/Acritarchs/Prasinophytes,Acanthaulax granulata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dinoflagellates/Acritarchs/Prasinophytes,Acanthaulax wilsonii,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
PI_df = pd.read_csv(input_file, dtype=str )

log_df(PI_df)
# (8906, 40)

(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
add_genus_species(PI_df)
log_df(PI_df)

(8906, 41)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus species
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aandalusiella ivoirensis
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abratopdinium cardioforme


In [64]:
species_df = PI_df[PI_df['genus species'].notna()].copy()[['taxon_group',  'genus species']]
species_df.drop_duplicates(inplace=True)

log_df(species_df)
# (5241, 2)

(5241, 2)


,taxon_group,genus species
3,dinoflagellates,Aandalusiella ivoirensis
4,dinoflagellates,Abratopdinium cardioforme
5,dinoflagellates,Abratopdinium kerguelense
6,dinoflagellates,Acanthaulax granulata
7,dinoflagellates,Acanthaulax wilsonii


In [82]:
merge_df = species_df.merge(old_species_df, how='outer', on='genus species', 
                            indicator=True, suffixes=(None, '_y'))
merge_df['check'] = False

log_df(merge_df)

(5769, 22)


,taxon_group,genus species,taxon_group_y,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,_merge,check
0,dinoflagellates,Aandalusiella ivoirensis,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,False
1,dinoflagellates,Abratopdinium cardioforme,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,False
2,dinoflagellates,Abratopdinium kerguelense,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,False
3,dinoflagellates,Acanthaulax granulata,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,False
4,dinoflagellates,Acanthaulax wilsonii,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,False


In [83]:
merge_df['_merge'].value_counts()

both          5155
right_only     423
left_only      191
Name: _merge, dtype: int64

In [91]:
filter_df = merge_df[(merge_df['_merge'] == 'both') | (merge_df['_merge'] == 'left_only')]
log_df(filter_df)

(5346, 22)


,taxon_group,genus species,taxon_group_y,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,family_taxon_name,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,_merge,check
0,dinoflagellates,Aandalusiella ivoirensis,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
1,dinoflagellates,Abratopdinium cardioforme,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
2,dinoflagellates,Abratopdinium kerguelense,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
3,dinoflagellates,Acanthaulax granulata,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
4,dinoflagellates,Acanthaulax wilsonii,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True


In [85]:
pbdb.fetch_pdbd_data(filter_df, 'genus species')

0 50 100 150 200 250 300 400 450 500 650 750 850 900 950 1000 1150 1200 1250 1300 1350 1400 1500 1550 1650 1700 1750 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3450 3600 3700 3750 3800 3850 3900 3950 4050 4100 4200 4250 4300 4400 4450 4500 4550 4600 4700 4800 4850 4900 4950 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 

In [92]:
filter_df.to_csv(species_path)

## add species pbdb info to unapproved taxa 

In [93]:
species_df = pd.read_csv(species_path, dtype=str)

log_df(species_df)
# (5346, 23)

(5346, 23)


,Unnamed: 0,taxon_group,genus species,taxon_group_y,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,_merge,check
0,0,dinoflagellates,Aandalusiella ivoirensis,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
1,1,dinoflagellates,Abratopdinium cardioforme,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
2,2,dinoflagellates,Abratopdinium kerguelense,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
3,3,dinoflagellates,Acanthaulax granulata,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True
4,4,dinoflagellates,Acanthaulax wilsonii,Dinoflagellates/Acritarchs/Prasinophytes,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,both,True


In [94]:
species_df = species_df[species_df['pbdb_taxon_name'].notna() & (species_df['pbdb_taxon_rank'] == 'species')]
log_df(species_df)
#  (1025, 23)

(1025, 23)


,Unnamed: 0,taxon_group,genus species,taxon_group_y,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,genus_taxon_id,genus_taxon_name,family_taxon_id,...,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,_merge,check
5,5,dinoflagellates,Achilleodinium bianii,Dinoflagellates/Acritarchs/Prasinophytes,323992,Achilleodinium bianii,species,323991,Achilleodinium,321603,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,both,False
14,14,dinoflagellates,Achomosphaera ramulifera,Dinoflagellates/Acritarchs/Prasinophytes,277049,Achomosphaera ramulifera,species,277048,Achomosphaera,321603,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,both,False
15,15,dinoflagellates,Achomosphaera sagena,Dinoflagellates/Acritarchs/Prasinophytes,323552,Achomosphaera sagena,species,277048,Achomosphaera,321603,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,both,False
16,16,dinoflagellates,Achomosphaera triangulata,Dinoflagellates/Acritarchs/Prasinophytes,323940,Achomosphaera triangulata,species,277048,Achomosphaera,321603,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,both,False
17,17,dinoflagellates,Achomosphaera verdieri,Dinoflagellates/Acritarchs/Prasinophytes,323553,Achomosphaera verdieri,species,277048,Achomosphaera,321603,...,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,both,False


In [99]:
PI_df = pd.read_csv(input_file, dtype=str )
add_genus_species(PI_df)

log_df(PI_df)
# (8906, 40)


(8906, 41)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,genus species
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aandalusiella ivoirensis
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abratopdinium cardioforme


In [103]:
pbdb.add_pbdb_data(PI_df, species_df, 'genus species')
del PI_df['genus species']

In [104]:
 set(species_df.columns) - set(PI_df.columns)

{'_merge', 'check', 'genus species', 'taxon_group_y'}

In [105]:
PI_df.to_csv(input_file, index=False)

## update genera csv

In [115]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)

(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
old_genus_df =  pd.read_csv(old_genus_path, dtype=str)
old_genus_df['check'] = True
del old_genus_df['taxon_group'] 

log_df(old_genus_df)

(2537, 21)


,genus name,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,check
0,Labyrinthodinium,443826,Labyrinthodinium,genus,NaN,NaN,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,Maduradinium,325673,Maduradinium,genus,277915,Peridiniaceae,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,Pyxidiella,336773,Pyxidiella,genus,277915,Peridiniaceae,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,Aandalusiella,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,Abratopdinium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True


In [143]:
genus_df = df[df['genus name'].notna()].copy()[['taxon_group', 'genus name']]
genus_df.drop_duplicates(inplace=True)
genus_df['check'] = False

log_df(genus_df)

(2313, 3)


,taxon_group,genus name,check
0,dinoflagellates,Labyrinthodinium,False
1,dinoflagellates,Maduradinium,False
2,dinoflagellates,Pyxidiella,False
3,dinoflagellates,Aandalusiella,False
4,dinoflagellates,Abratopdinium,False


In [144]:
merge_df = genus_df.merge(old_genus_df, how='left', on='genus name',
                         indicator=True, suffixes=(None, '_y'))
merge_df.drop_duplicates(subset=['taxon_group', 'genus name'], inplace=True)
merge_df.loc[merge_df['pbdb_taxon_id'].notna(), 'check'] = True

log_df(merge_df)

(2313, 24)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,...,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name,check_y,_merge
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,NaN,NaN,321606,Gonyaulacales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,both
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277915,Peridiniaceae,277919,Peridiniales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,both
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277915,Peridiniaceae,277919,Peridiniales,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,both
3,dinoflagellates,Aandalusiella,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,both
4,dinoflagellates,Abratopdinium,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,both


check for duplicate genus

In [145]:
genus_df[genus_df.duplicated(subset=['taxon_group', 'genus name'])]

,taxon_group,genus name,check


In [147]:
pbdb.fetch_pdbd_data(genus_df, 'genus name')


0 500 550 900 950 1250 1350 1450 1550 1600 1650 1750 2250 2850 3000 3100 3950 4050 4150 4200 4250 4300 4500 4600 4850 5100 5200 5500 5550 5650 5950 6050 6100 6850 7000 7150 7950 8150 8250 8400 8450 8850 8900 

In [148]:
log_df(genus_df)

(2313, 22)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_id,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella,True,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium,True,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [149]:
genus_df[genus_df['pbdb_taxon_id'].isna()].shape

(511, 22)

In [150]:
genus_df.drop_duplicates(inplace=True)

log_df(genus_df)

(2313, 22)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_id,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella,True,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium,True,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [153]:
genus_df.to_csv(genus_path, index=False)

## update genus csv with no higher pbdb ranks
refetch pbdb infor for higher taxa that don't have pbdb data

In [154]:
df = pd.read_csv(genus_path, dtype=str)

temp_df = df[
    df['unranked clade_taxon_id'].isna() &
    df['kingdom_taxon_id'].isna() &
    df['phylum_taxon_id'].isna() &
    df['family_taxon_id'].isna() &
    df['order_taxon_id'].isna() &
    df['class_taxon_id'].isna() &
    df['pbdb_taxon_id'].notna() 
].copy()
log_df(temp_df)


(22, 22)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_id,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
5,dinoflagellates,Acanthaulax,True,276986,Acanthaulax,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,dinoflagellates,Alterbidinium,True,276891,Alterbidinium,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,dinoflagellates,Caligodinium,True,276897,Caligodinium,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,dinoflagellates,Conosphaeridium,True,277189,Conosphaeridium,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
118,dinoflagellates,Endoceratium,True,277207,Endoceratium,genus,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [155]:
for index, row in temp_df.iterrows():  
        
    time.sleep(0.5)
        
    url =  PBDB_TAXA_ID + str(row['pbdb_taxon_id'])
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()["records"]
        if len(data) == 1:
            rank = data[0]["taxon_rank"]
            df.at[index, f'pbdb_taxon_id'] = str(data[0]["taxon_no"])
            df.at[index, f'pbdb_taxon_name'] = data[0]["taxon_name"]
            df.at[index, f'pbdb_taxon_rank'] = data[0]["taxon_rank"]
               
            round = 0
            get_parent_taxa(df, data[0]["parent_no"], data[0]["taxon_rank"], round, index, None)
        else:
            print(name, len(data), ' found')
    else:
        print(name, ' not found')

In [157]:
df.to_csv(genus_path, index=False)

## add genus pbdb to input_file

In [158]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)


(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
genus_df= pd.read_csv(genus_path, dtype=str)

log_df(genus_df)
# (2313, 22)

(2313, 22)


,taxon_group,genus name,check,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,...,phylum_taxon_id,phylum_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,kingdom_taxon_id,kingdom_taxon_name,subclass_taxon_id,subclass_taxon_name,genus_taxon_id,genus_taxon_name
0,dinoflagellates,Labyrinthodinium,True,443826,Labyrinthodinium,genus,321606,Gonyaulacales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,Maduradinium,True,325673,Maduradinium,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,Pyxidiella,True,336773,Pyxidiella,genus,277919,Peridiniales,321578,Dinophyceae,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [160]:
pbdb.add_pbdb_data(df, genus_df, 'genus name')

In [161]:
log_df(df)
# (8906, 40)

(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [162]:
df.to_csv(input_file, index=False)

## create higher taxa csv 

In [185]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)

(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


check for for taxon that has both higher taxa and genus name 

In [186]:
df[df['Any taxon above genus'].notna() & df['genus name'].notna()]

,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name


In [187]:
higher_df =  pbdb.create_higher_taxa_df(df)
higher_df['check'] = False
log_df(higher_df)
# 69

(65, 16)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
31,dinoflagellates,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
376,dinoflagellates,Dinoflagellata indet.,52776,Genasauria,unranked clade,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
1019,dinoflagellates,Prasinophyta indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1324,palynology,Angiospermae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
1337,palynology,Asteraceae indet.,183083,Asteraceae,family,183083,Asteraceae,54943,Asterales,55217,Magnoliopsida,55350,Spermatophyta,54311,Plantae,False


In [188]:
higher_df[higher_df.duplicated(subset=['taxon_group', 'Any taxon above genus'])]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check


In [189]:
higher_df[higher_df.duplicated(subset=['Any taxon above genus'])]

,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
1349,palynology,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
5543,nannofossils,Dinoflagellata indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [190]:
pbdb.fetch_pdbd_data(higher_df, 'Any taxon above genus')

In [191]:
higher_df.drop_duplicates(inplace=True)

log_df(higher_df)


(65, 16)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
31,dinoflagellates,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
376,dinoflagellates,Dinoflagellata indet.,52776,Genasauria,unranked clade,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
1019,dinoflagellates,Prasinophyta indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1324,palynology,Angiospermae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1337,palynology,Asteraceae indet.,183083,Asteraceae,family,183083,Asteraceae,54943,Asterales,55217,Magnoliopsida,55350,Spermatophyta,54311,Plantae,False


In [192]:
higher_df.columns

Index(['taxon_group', 'Any taxon above genus', 'pbdb_taxon_id',
       'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id',
       'family_taxon_name', 'order_taxon_id', 'order_taxon_name',
       'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id',
       'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name', 'check'],
      dtype='object')

In [193]:
higher_df.to_csv(higher_path, index=False)

## add higher taxa pbdb to input file

In [194]:
df = pd.read_csv(input_file, dtype=str)
log_df(df)
# (8906, 40)

(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [195]:
higher_df = pd.read_csv(higher_path, dtype=str)
log_df(higher_df)
# 75

(65, 16)


,taxon_group,Any taxon above genus,pbdb_taxon_id,pbdb_taxon_name,pbdb_taxon_rank,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,check
0,dinoflagellates,Acritarcha indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,dinoflagellates,Dinoflagellata indet.,52776,Genasauria,unranked clade,NaN,NaN,NaN,NaN,36322,Reptilia,NaN,NaN,NaN,NaN,False
2,dinoflagellates,Prasinophyta indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
3,palynology,Angiospermae indet.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
4,palynology,Asteraceae indet.,183083,Asteraceae,family,183083,Asteraceae,54943,Asterales,55217,Magnoliopsida,55350,Spermatophyta,54311,Plantae,False


In [196]:
pbdb.add_pbdb_data(df, higher_df, 'Any taxon above genus')

In [197]:
log_df(df)

(8906, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [198]:
df.to_csv(input_file, index=False)

## QA taxa

In [201]:
df = pd.read_csv(input_file, dtype=str, nrows=2441)
log_df(df)

(2441, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
filter_df = df[df['pbdb_taxon_id'].isna()]
log_df(filter_df)

(258, 40)


,Unnamed: 0,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
3,3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,dinoflagellates,Abratopdinium kerguelense,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,31,dinoflagellates,Acritarch forma A,Acritarcha indet.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,Acritarcha indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,32,dinoflagellates,Acritarch forma B,Acritarcha indet.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,Acritarcha indet.,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
genus_df = filter_df[['genus name']].copy()
genus_df.drop_duplicates(inplace=True)
genus_df.dropna(inplace=True)
genus_df['check'] = False
log_df(genus_df)

(103, 2)


,genus name,check
3,Aandalusiella,False
4,Abratopdinium,False
40,Aireiana,False
44,Algidasphaeridium,False
61,Alisogymnium,False


In [214]:
pbdb.fetch_pdbd_data(genus_df, 'genus name')

500 

In [215]:
log_df(genus_df)

(103, 3)


,genus name,check,pbdb_taxon_id
3,Aandalusiella,True,<NA>
4,Abratopdinium,True,<NA>
40,Aireiana,True,<NA>
44,Algidasphaeridium,True,<NA>
61,Alisogymnium,True,<NA>


## reorder columns

In [219]:
df = pd.read_csv(input_file, dtype=str)
del df['Unnamed: 0']
log_df(df)

(8906, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [220]:
PI_df = pd.read_csv(initial_input_file, dtype=str)

log_df(PI_df)

(8906, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,subclass_taxon_id,subclass_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,dinoflagellates,?Labyrinthodinium sp. 1,? Labyrinthodinium sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Labyrinthodinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
1,dinoflagellates,?Maduradinium sp.,? Maduradinium sp.,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Maduradinium,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
2,dinoflagellates,?Pyxidiella sp. 1,? Pyxidiella sp. 1,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,?,Pyxidiella,NaN,...,NaN,NaN,321578,Dinophyceae,NaN,NaN,NaN,NaN,NaN,NaN
3,dinoflagellates,Aandalusiella ivoirensis,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Aandalusiella,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,dinoflagellates,Abratopdinium cardioforme,NaN,NaN,"Updated taxon_group from ""Dinoflagellates/Acri...",NaN,NaN,NaN,Abratopdinium,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [221]:
PI_df.columns == df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])

### ========================================
## stop
### ========================================


## create crosswalk file

In [96]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [97]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [98]:
crosswalk_df = nt.create_taxa_crosswalk_df(df)
log_df(crosswalk_df)
# 7763
# 7760
# 7759

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'verbatim_name', 'name comment field', 'Comment', 'Notes (change to Internal only notes?)', 'comments']
initial df:  (7763, 17)
remove nontaxa df:  (7760, 17)
drop duplicates df:  (7759, 17)
(7759, 17)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,normalized_name,taxon_group,verbatim_name,name comment field,Comment,Notes (change to Internal only notes?),comments
0,,,Abyssamina,,,,incisa,,,,Abyssamina incisa,benthic_forams,Abyssamina incisa,NaN,,,
1,,,Adercotryma,,,,glomeratum,,,,Adercotryma glomeratum,benthic_forams,Adercotryma glomeratum,NaN,,,
2,,,Adercotryma,,,,sp.,,,,Adercotryma sp.,benthic_forams,Adercotryma sp.,NaN,,,
3,,,Alabamina,,,,decorata,,,,Alabamina decorata,benthic_forams,Alabamina decorata,NaN,,,
4,,,Alabamina,,,,haitiensis,,,,Alabamina haitiensis,benthic_forams,Alabamina haitiensis,NaN,,,


In [99]:
PI_df.to_csv(crosswalk_file, index=False)

## create taxa file

In [100]:
PI_df = pd.read_csv(input_file, dtype=str)
log_df(PI_df)
# 7763

(7763, 39)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN


In [101]:
df = nt.add_normalized_name_column(PI_df)
log_df(df)

(7763, 40)


,taxon_group,verbatim_name,name,Comment,Notes (change to Internal only notes?),Any taxon above genus modifier,Any taxon above genus,genus modifier,genus name,subgenera modifier,...,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name,unranked clade_taxon_id,unranked clade_taxon_name,normalized_name
0,benthic_forams,Abyssamina incisa,NaN,NaN,NaN,NaN,NaN,NaN,Abyssamina,NaN,...,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Abyssamina incisa
1,benthic_forams,Adercotryma glomeratum,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma glomeratum
2,benthic_forams,Adercotryma sp.,NaN,NaN,NaN,NaN,NaN,NaN,Adercotryma,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Adercotryma sp.
3,benthic_forams,Alabamina decorata,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina decorata
4,benthic_forams,Alabamina haitiensis,NaN,NaN,NaN,NaN,NaN,NaN,Alabamina,NaN,...,NaN,NaN,NaN,288974,Foraminifera,212476,Rhizaria,NaN,NaN,Alabamina haitiensis


In [102]:
taxa_df = nt.create_taxa_list_df(df)
log_df(taxa_df)
# 7763
# 7760
# 7658

fields: ['Any taxon above genus', 'genus modifier', 'genus name', 'subgenera modifier', 'subgenera name', 'species modifier', 'species name', 'subspecies modifier', 'subspecies name', 'non-taxa descriptor', 'normalized_name', 'taxon_group', 'pbdb_taxon_id', 'pbdb_taxon_name', 'pbdb_taxon_rank', 'family_taxon_id', 'family_taxon_name', 'order_taxon_id', 'order_taxon_name', 'class_taxon_id', 'class_taxon_name', 'phylum_taxon_id', 'phylum_taxon_name', 'kingdom_taxon_id', 'kingdom_taxon_name']
initial df:  (7763, 25)
remove nontaxa df:  (7760, 25)
drop duplicates df:  (7658, 25)
(7658, 25)


,Any taxon above genus,genus modifier,genus name,subgenera modifier,subgenera name,species modifier,species name,subspecies modifier,subspecies name,non-taxa descriptor,...,family_taxon_id,family_taxon_name,order_taxon_id,order_taxon_name,class_taxon_id,class_taxon_name,phylum_taxon_id,phylum_taxon_name,kingdom_taxon_id,kingdom_taxon_name
0,,,Abyssamina,,,,incisa,,,,...,103796,Chilostomellidae,279579,Rotaliida,428504,Globothalamea,288974,Foraminifera,212476,Rhizaria
1,,,Adercotryma,,,,glomeratum,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
2,,,Adercotryma,,,,sp.,,,,...,,,,,,,288974,Foraminifera,212476,Rhizaria
3,,,Alabamina,,,,decorata,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria
4,,,Alabamina,,,,haitiensis,,,,...,82213,Alabaminidae,,,,,288974,Foraminifera,212476,Rhizaria


In [103]:
taxa_df.to_csv(taxa_file, index=False)